In [9]:
!pip install spacy
!python -m spacy download en_core_web_sm

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('punkt')  # Download the punkt tokenizer data
nltk.download('stopwords')  # Download the stopwords data



C:\Users\Sabina\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sabina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sabina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
content = pd.read_csv("content_translated_final.csv")
topics = pd.read_csv("topics_final.csv")

In [3]:
content

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Add multi-digit numbers: 48,029+233,930","Add 48,029+233,930 using the standard algorithm.",video,NaN,es,NaN,NaN
1,c_000087304a9e,Finding the factors of a number,Sal finds the factors of 120.,video,NaN,it,NaN,NaN
2,c_0000ad142ddb,Add demand curves,How to Add Demand Curves,video,NaN,es,NaN,NaN
3,c_0000c03adc8d,approach swim,In this video you will learn the approach swim...,document,approach swim\nKnowing how to swim in the wave...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,New York State Common Core Standards\n\nmath s...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...,...
154042,c_fffcbdd4de8b,2. 12: Diffusion,in,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA
154043,c_fffe15a2d069,Add up by making groups of 10,Sal adds 5+68 by breaking the 5 into a 2 and a 3.,video,NaN,it,NaN,NaN
154044,c_fffed7b0d13a,Introduction to subtraction,Sal talks about what it means to subtract. The...,video,NaN,pt,NaN,NaN
154045,c_ffff04ba7ac7,SA of a Cone,in,video,NaN,en,NaN,NaN


In [4]:
topics

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,The detection of resistors,Investigating materials that cause a reduction...,000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,in,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Inputs and outputs of a function,Understand a little more about functions.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,in,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Graphs of Exponential Functions (Algebra 2 Level),Learn more about graphing complex exponential ...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,in,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
76969,t_fffe14f1be1e,Lesson 7,in,6e90a7,aligned,6,es,t_d448c707984d,True
76970,t_fffe811a6da9,Determine the relationship between polar coord...,"5 Bassakhs, 1st floor, 0499, Yasf",9fd860,source,2,ar,t_5b4f3ba4eb7d,True


In [42]:
# initialize stemmer
stemmer = PorterStemmer()

# preprocessing of content dataset

In [43]:
# replace NaN values with an empty string
content['title'] = content['title'].fillna('')
content['description'] = content['description'].fillna('')
content['text'] = content['text'].fillna('')

In [44]:
# apply stemming to title, description, and text columns of content
stem_columns = ["title", "description", "text"]
for col in stem_columns:
    content[col] = content[col].apply(lambda text: ' '.join([stemmer.stem(word) for word in word_tokenize(text)]))


In [49]:
# removing stopwords
stop_words = set(stopwords.words('english'))  # Set of English stop words
content['title'] = content['title'].apply(lambda x: ' '.join([word for word in word_tokenize(str(x).lower()) if word.isalnum() and word not in stop_words]))
content['description'] = content['description'].apply(lambda x: ' '.join([word for word in word_tokenize(str(x).lower()) if word.isalnum() and word not in stop_words]))

In [50]:
content['text'] = content['text'].apply(lambda x: ' '.join([word for word in word_tokenize(str(x).lower()) if word.isalnum() and word not in stop_words]))

# preprocessing of topic dataset

In [71]:
# replace NaN values with an empty string
topics['title'] = topics['title'].fillna('')
topics['description'] = topics['description'].fillna('')

In [72]:
# apply stemming to title, and description columns of topics
stem_columns = ["title", "description"]
for col in stem_columns:
    topics[col] = topics[col].apply(lambda text: ' '.join([stemmer.stem(word) for word in word_tokenize(text)]))


In [67]:
# removing stopwords
topics['title'] = topics['title'].apply(lambda x: ' '.join([word for word in word_tokenize(str(x).lower()) if word.isalnum() and word not in stop_words]))
topics['description'] = topics['description'].apply(lambda x: ' '.join([word for word in word_tokenize(str(x).lower()) if word.isalnum() and word not in stop_words]))

In [72]:
# save preprocessed topics and content
topics.to_csv('cleared_topics.csv', index=False)
content.to_csv('cleared_content.csv', index=False)